# Inleiding

Voor deze opdracht gaan jullie zelf op zoek naar je eigen dataset. De dataset bevat in ieder geval een kolom met een timestamp (een datum of een tijdstip) met gelijke stapgrootte (uurlijkse/dagelijkse/jaarlijkse data). Daarnaast komt deze tijdaanduiding minstens 36x terug, dus bij het gebruik van maanden minstens 36 maanden. Liever heb je een dataset die groot genoeg is om de train/test splitsing logisch te maken. Denk aan een seizoenseffect dat pas eind van elk jaar gebeurt: dan zou je je testset niet de laatste paar maanden van dat jaar zijn, omdat het zo anders van je trainset zou zijn. 

Let op: In tegenstelling tot de eerste opdracht, voor deze en de resterende 1 opdracht worden jullie peer reviews alleen beoordeeld op een ja of nee basis. 

Zoals geschreven in de vakbeschrijving:

"Het niet op tijd of lui maken van een individuele peer review resulteert in een reductie van 0,5 punt voor het cijfer van de bijhorende opdracht van de reviewer."

Gezien dat jullie peer reviews zijn sowieso kwalitatief beoordeeld voor Opdracht 1, is deze halfpunt reductie alleen voor Opdrachten 2 t/m 4 geldig. 

### Algemene Coderen Conventies

De volgorde van het python script is belangrijk. Zo zorg je bijvoorbeeld altijd dat eerst de benodigde packages worden ingeladen geïnstalleerd en geïmporteerd). Verder is het goed om van tevoren jouw working directory te controleren. 

Schrijf het python script daarom met een duidelijke indeling van met verschillende stappen. Bij elke stap dient toelichting worden gegeven:
1. Wat is het doel van deze stap?
2. Wat is de input van deze stap?
3. Wat is de output van deze stap en waar heb je deze output hierna nog nodig?

Hieronder voorbeelden van de verschillende punten hierboven.

In [ ]:
### Voorbeeld: Standaard Imports
import numpy as np
import matplotlib.pyplot as plt


### Controleer Working Directory
import os
print(os.getcwd())

# Pas je working directory aan indien gewenst
path = 'C:\\Users\\rburrus\\Desktop\\Jupyter Notebook' # zet hier wat voor jou goed is
os.chdir(path)
print(os.getcwd())


### Voorbeeld: Functie met Beschrijving Schrijven

def voorbeeld_functie(parameter1, parameter2 = 2):
    # Doel: dit functie maakt een bepaalde berekening
    # In: parameter1 (float), parameter2 (integer)
    # Uit: parameter1 macht parameter2 (standaard macht 2)
    
    return parameter1 ** parameter2

# Theorievragen

Beantwoord voordat je met de dataset aan de slag gaan, eerst de volgende vragen:

Dit is wat er nodig is voor de hoogste score volgens de beoordelingsrubriek:

"Heeft de theorie vragen beantwoord en de uitleg is begrijpelijk en er worden voorbeelden bij gehaald. De uitleg wordt ondersteund door visualisaties. Ook wordt autocorrelatie beschreven bij de uiteindelijke modellen."

Wat is autocorrelatie?

Let het verschil uit tussen een AR model en een MA model.

Hoe werkt een ARIMA model om deze twee methoden te combineren en wat doet het "I" element?

Geef een korte uitleg over de techniek Exponential Smoothing.

# Deelopdracht 1/3 - Dataverkenning

Splits de data op in een train en test dataset. De traindataset mag niet te klein zijn. Doe de splitsing op chronologische volgorde. Bij bijvoorbeeld januari tot en met december, zouden november en december de testdataset zijn (je kan alleen vooruit in de tijd).

Dit is wat er nodig is voor de hoogste score volgens de beoordelingsrubriek:

"Heeft verkennende plots gemaakt. Heeft datatype van tijdsvariabele correct behandeld en gekeken of de tijdreeks geen gaten heeft. Daarnaast is er een goede analyse van trend, seizoenspatronen en autocorrelatie gemaakt. De gevonden patronen worden bediscussieerd en er worden mogelijke verklaringen/interpretaties voor gegeven."

In [ ]:
(code cell)

Voer een uitgebreidere dataverkenning uit (volgens de beschrijving hierboven uit de beoordelingsrubriek) op jullie gekozen dataset. Gebruik zoveel cellen als nodig, maar liever alleen plotjes/prints maken die echt een impact op jullie modellen gaan maken (samen met uitleg tekst van wat jullie daarmee opvalt).  

In [ ]:
(code cell)

Is er een trend zichtbaar in de data? Wat walt je hier op?

(Markdown cell voor uitleg)

Is er een seizoenspatroon zichtbaar in de data? Wat valt je hier op?

(Markdown cell voor uitleg)

Zit er autocorrelatie in de data? Wat valt je hier op?

(Markdown cell voor uitleg)

# Deelopdracht 2/3 - Prophet Package

Maak een voorspelling met het Prophet package.

Gebruik ‘conda install -c conda-forge prophet’ 

Check even de installatie instructies op deze pagina: https://facebook.github.io/prophet/docs/quick_start.html

Dit is wat er nodig is voor de hoogste score volgens de beoordelingsrubriek:

"De voorspelling met het Prophet package is correct uitgevoerd en uitgebreid beschreven. De prestatiemetrics worden op train en test berekend. Er wordt geëxperimenteerd met verschillende instellingen."

Visualiseer de voorspelling.

In [ ]:
(code cell)

Wat valt je op?

(Markdown cell voor uitleg)

Bereken een kwaliteitsmaat (bijvoorbeeld RMSE of MAE) op de train- en testdataset.

In [ ]:
(code cell)

Wat valt je hier op?

(Markdown cell voor uitleg)

# Deelopdracht 3/3 - Gekozen Model

Maak een voorspelling met het statsmodels package op de traindataset. Kies een optie uit de volgende opties:

1. AR model
2. MA model
3. ARIMA
4. Exponential Smoothing

Zie deze link voor meer informatie: https://www.statsmodels.org/stable/user-guide.html#time-series-analysis

Dit is wat er nodig is voor de hoogste score volgens de beoordelingsrubriek:

"De voorspelling met het statsmodels package is correct uitgevoerd en uitgebreid beschreven. De prestatiemetrics worden op train en test berekend. Er wordt geëxperimenteerd met verschillende instellingen en de verschillen met de resultaten uit het prophet package worden bediscussieerd."

Visualiseer de voorspelling.

In [ ]:
(code cell)

Wat valt je hier op?

(Markdown cell voor uitleg)

Bereken dezelfde kwaliteitsmaat (bijvoorbeeld RMSE of MAE) als bij de Prophet package hierboven op de train- en testdataset.

In [ ]:
(code cell)

(Markdown cell voor uitleg)

Vergelijk de voorspellingen (dus van de Prophet package en hier met jullie gekozen model). Welke van de twee komt het meest overeen met de testdataset?

In [ ]:
(code cell)

(Markdown cell voor uitleg)

Voor welke van de twee modellen zou je kiezen als je echt een voorspelling zou doen? Mochten beiden niet goed werken, wat voor ander model zou je dan proberen?

(Markdown cell voor uitleg)

Wat kan je zeggen over autocorrelatie in je definitieve model? 

In [ ]:
(code cell)

(Markdown cell voor uitleg)